# Suspicious Typical Malware Back Connect Ports
Detects programs that connect to typical malware back connect ports based on statistical analysis from two different sandbox system databases

## Rule Content
```
- title: Suspicious Typical Malware Back Connect Ports
  id: 4b89abaa-99fe-4232-afdd-8f9aa4d20382
  status: experimental
  description: Detects programs that connect to typical malware back connect ports
    based on statistical analysis from two different sandbox system databases
  references:
  - https://docs.google.com/spreadsheets/d/17pSTDNpa0sf6pHeRhusvWG6rThciE8CsXTSlDUAZDyo
  author: Florian Roth
  date: 2017/03/19
  tags:
  - attack.command_and_control
  - attack.t1043
  logsource:
    product: windows
    service: sysmon
    definition: 'Use the following config to generate the necessary Event ID 10 Process
      Access events: <ProcessAccess onmatch="include"><CallTrace condition="contains">VBE7.DLL</CallTrace></ProcessAccess><ProcessAccess
      onmatch="exclude"><CallTrace condition="excludes">UNKNOWN</CallTrace></ProcessAccess>'
    category: null
  detection:
    selection:
      EventID: 3
      Initiated: 'true'
      DestinationPort:
      - '4443'
      - '2448'
      - '8143'
      - '1777'
      - '1443'
      - '243'
      - '65535'
      - '13506'
      - '3360'
      - '200'
      - '198'
      - '49180'
      - '13507'
      - '6625'
      - '4444'
      - '4438'
      - '1904'
      - '13505'
      - '13504'
      - '12102'
      - '9631'
      - '5445'
      - '2443'
      - '777'
      - '13394'
      - '13145'
      - '12103'
      - '5552'
      - '3939'
      - '3675'
      - '666'
      - '473'
      - '5649'
      - '4455'
      - '4433'
      - '1817'
      - '100'
      - '65520'
      - '1960'
      - '1515'
      - '743'
      - '700'
      - '14154'
      - '14103'
      - '14102'
      - '12322'
      - '10101'
      - '7210'
      - '4040'
      - '9943'
    filter1:
      Image: '*\Program Files*'
    filter2:
      DestinationIp:
      - 10.*
      - 192.168.*
      - 172.16.*
      - 172.17.*
      - 172.18.*
      - 172.19.*
      - 172.20.*
      - 172.21.*
      - 172.22.*
      - 172.23.*
      - 172.24.*
      - 172.25.*
      - 172.26.*
      - 172.27.*
      - 172.28.*
      - 172.29.*
      - 172.30.*
      - 172.31.*
      - 127.*
      DestinationIsIpv6: 'false'
    condition: selection and not ( filter1 or filter2 )
  falsepositives:
  - unknown
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((event_id:"3" AND network_initiated:"true" AND dst_port:("4443" OR "2448" OR "8143" OR "1777" OR "1443" OR "243" OR "65535" OR "13506" OR "3360" OR "200" OR "198" OR "49180" OR "13507" OR "6625" OR "4444" OR "4438" OR "1904" OR "13505" OR "13504" OR "12102" OR "9631" OR "5445" OR "2443" OR "777" OR "13394" OR "13145" OR "12103" OR "5552" OR "3939" OR "3675" OR "666" OR "473" OR "5649" OR "4455" OR "4433" OR "1817" OR "100" OR "65520" OR "1960" OR "1515" OR "743" OR "700" OR "14154" OR "14103" OR "14102" OR "12322" OR "10101" OR "7210" OR "4040" OR "9943")) AND (NOT ((process_path.keyword:*\\Program\ Files* OR (dst_ip_addr.keyword:(10.* OR 192.168.* OR 172.16.* OR 172.17.* OR 172.18.* OR 172.19.* OR 172.20.* OR 172.21.* OR 172.22.* OR 172.23.* OR 172.24.* OR 172.25.* OR 172.26.* OR 172.27.* OR 172.28.* OR 172.29.* OR 172.30.* OR 172.31.* OR 127.*) AND DestinationIsIpv6:"false")))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()